<a href="https://colab.research.google.com/github/alistairbryan/autonomous-parking-attendant/blob/master/License_Plate_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classify License Plates

This program trains deconstructs 4-character license plates and trains a neural network model to classify their characters.

In [0]:
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import h5py


# Helper libraries
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import math
import re
import cv2 as cv
import string
from collections import Counter
from PIL import Image
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.models import model_from_json


print(tf.__version__)

## Declare Constants and Initialize Data Augmenter

In [0]:
Y_OFFSET = 5
FIRST_OFFSET = -203
SECOND_OFFSET = -103
THIRD_OFFSET = 98
FOURTH_OFFSET = 197

# Create data generator
datagen = ImageDataGenerator(width_shift_range=0.3, height_shift_range=0.1, rotation_range=5, brightness_range=[0.3,0.9], zoom_range=[1.0, 1.3])
datagen_parking = ImageDataGenerator(width_shift_range=0.3, height_shift_range=0.3, rotation_range=10, brightness_range=[0.3,0.9], zoom_range=[1.0, 1.6])

## Authorize Google Drive Access

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Load License Plate Data

In [0]:
# Training Data
PATH = "/content/drive/My Drive/ENPH 353/Lab 4 - Alistair/Training_Pictures/"
oldTrainingLabels = !ls "{PATH}"
trainingLabels = []
trainingFiles = []
for label in oldTrainingLabels:
  label = label.split("\t")
  for l in label:
    trainingFiles.append(l)
    trainingLabels.append(l[6:10])
print(trainingFiles)
print(trainingLabels)

# Testing Data
PATH_TESTING = "/content/drive/My Drive/ENPH 353/Lab 4 - Alistair/Testing_Pictures/"
oldTestingLabels = !ls "{PATH_TESTING}"
testingLabels = []
testingFiles = []
for label in oldTestingLabels:
  label = label.split("\t")
  for l in label:
    testingFiles.append(l)
    testingLabels.append(l[6:10])

In [0]:
# Single Characters
PATH_CHARS = "/content/drive/My Drive/ENPH 353/Lab 4 - Alistair/comp_img/"
oldCharLabels = !ls "{PATH_CHARS}"
singleCharLabels = []
charFiles = []
labs = []
almost_labels = []
for label in oldCharLabels:
  label = label.split("\t")
  for l in label:
    singleCharLabels.append(l[1])
    singleCharLabels.append(l[13])
    charFiles.append(l[1:9])
    charFiles.append(l[13:21])

print(singleCharLabels)
print(charFiles)

In [0]:

trainingSet = []

# Populate training set
for file in trainingFiles: trainingSet.append(cv.imread('%s%s' % (PATH, file)))
  

# Images loaded in rgb
trainingPlates = np.asarray(trainingSet)
print(trainingPlates.shape)

In [0]:
testingSet = []

for fileTest in testingFiles: testingSet.append(cv.imread('%s%s' % (PATH_TESTING, fileTest)))
testingPlates = np.asarray(testingSet)
print(testingPlates.shape)

In [0]:
charSet = []

for fileChar in charFiles: charSet.append(cv.imread('%s%s' % (PATH_CHARS, fileChar)))

# Images loaded in rgb
trainingChars = np.asarray(charSet)
print(trainingChars.shape)

## Show an Image

In [0]:
plt.figure()
plt.imshow(trainingChars[1])
plt.grid(False)
plt.show()

## Slice License Plates and Labels into Individual Characters

In [0]:
def crop(img, charNum):
    cropx = 90
    cropy = 298
    
    if (charNum == 0): xOffset = FIRST_OFFSET
    if (charNum == 1): xOffset = SECOND_OFFSET
    if (charNum == 2): xOffset = THIRD_OFFSET
    if (charNum == 3): xOffset = FOURTH_OFFSET

    y,x,_ = img.shape
    startx = x//2-(cropx//2) + xOffset
    starty = y//2-(cropy//2) + Y_OFFSET
    return img[starty:starty+cropy,startx:startx+cropx]

unscaledChars = [] # Contains the image of the character
charLabels = [] # Contains the label that corresponds to each image

i = 0

while i < len(trainingPlates):
  j = 0
  while j < 4:
    unscaledChars.append(crop(trainingPlates[i], j))
    charLabels.append(trainingLabels[i][j])
    j += 1
  i += 1

i = 0

while i < len(trainingChars):
  unscaledChars.append(trainingChars[i])
  charLabels.append(singleCharLabels[i])
  i += 1
unscaledChars = np.asarray(unscaledChars)

unscaledTestingChars = [] # Contains the image of the character
testingCharLabels = [] # Contains the label that corresponds to each image


while i < len(testingPlates):
  j = 0
  while j < 4:
    unscaledTestingChars.append(crop(testingPlates[i], j))
    testingCharLabels.append(testingLabels[i][j])
    j += 1
  i += 1
unscaledTestingChars = np.asarray(unscaledTestingChars)


## Map Character Labels Array Indices

In [0]:
nums = ['0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9']
class_names = nums + list(string.ascii_uppercase)

charLabelNums = []
for charLabel in charLabels:
  k = 0
  while k < len(class_names):
    if charLabel == class_names[k]:
      charLabelNums.append(k)
      break
    k += 1
print(charLabelNums)

testingCharLabelNums = []
for charLabel in testingCharLabels:
  k = 0
  while k < len(class_names):
    if charLabel == class_names[k]:
      testingCharLabelNums.append(k)
      break
    k += 1
print(testingCharLabelNums)

## Scale pixel values from 0-255 to 0-1

Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the *training set* and the *testing set* be preprocessed in the same way:

In [0]:
chars = unscaledChars
scaledChars = []
for char in chars:
  sized_char = cv.resize(char, dsize=(128, 128), interpolation=cv.INTER_CUBIC)
  gray_char = cv.cvtColor(sized_char, cv.COLOR_RGB2GRAY) / 255.0

  scaledChars.append(cv.blur(gray_char, (15,15)))

testingChars = unscaledTestingChars
scaledTestingChars = []
for char in testingChars:
  char = cv.blur(cv.resize(char, dsize=(128, 128), interpolation=cv.INTER_CUBIC), (15, 15)) / 255.0
  scaledTestingChars.append(char)
  
train_images = np.asarray(scaledChars)
test_images = np.asarray(scaledTestingChars)

train_labels = np.asarray(charLabelNums)
test_labels = np.asarray(testingCharLabelNums)

To verify that the data is in the correct format and that you're ready to build and train the network, let's display the first 25 images from the *training set* and display the class name below each image.

Check parking number characters are present.

In [0]:
plt.figure()
plt.imshow(train_images[-13])
print(train_labels[-13])
plt.grid(False)
plt.show()

print(train_labels[-12:-1].size)
train_labels[-12:-1]

In [0]:
# Configure iterator

print(np.expand_dims(train_images, 3).shape)

one_hot = keras.utils.to_categorical(train_labels, 36)

it = datagen.flow(np.expand_dims(train_images, 3), one_hot)
it_parking = datagen.flow(np.expand_dims(train_images[-12:-1], 3), one_hot[-12:-1])

plt.figure(figsize=(10,10))
for i in range(9):
  # define subplot
  plt.subplot(330 + 1 + i)
  # generate batch of images
  batch = it.next()

  print(batch[0][0].shape)

  # convert to unsigned integers for viewing
  image = batch[0].astype('uint8') / 255.0
  # plot raw pixel data
  plt.imshow(np.squeeze(image[0], 2))
  plt.colorbar()

plt.show()

In [0]:
finished_images = []
finished_labels = []
for i in range(0, 500):
  j = 0
  next_batch = it.next()
  while j < next_batch[0].shape[0]:
    finished_images.append(next_batch[0][j])
    finished_labels.append(next_batch[1][j])
    j += 1

for i in range(0, 50):
  j = 0
  next_batch = it_parking.next()
  while j < next_batch[0].shape[0]:
    finished_images.append(next_batch[0][j])
    finished_labels.append(next_batch[1][j])
    j += 1

In [0]:
finished_labels_array = np.asarray(finished_labels)
finished_images_array = np.asarray(finished_images) / 255.0
print(finished_labels_array.shape)
print(finished_images_array.shape)
print(finished_labels_array[0])
plt.imshow(np.squeeze(finished_images_array[-1]))
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Set up the layers

The basic building block of a neural network is the *layer*. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [0]:
from keras import layers
from keras import models
from keras import optimizers

from keras.utils import plot_model
from keras import backend

In [0]:

conv_model = models.Sequential()
conv_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                             input_shape=(128, 128, 1)))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
conv_model.add(layers.MaxPooling2D((2, 2)))
conv_model.add(layers.Flatten())
conv_model.add(layers.Dropout(0.5))
conv_model.add(layers.Dense(512, activation='relu'))
conv_model.add(layers.Dense(36, activation='softmax'))

In [0]:
def reset_weights(model):
    session = backend.get_session()
    for layer in conv_model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

reset_weights(conv_model)

The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer is a 36-node *softmax* layer that returns an array of 36 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 36 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:

* *Loss function* —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* *Optimizer* —This is how the model is updated based on the data it sees and its loss function.
* *Metrics* —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [0]:
LEARNING_RATE = 1e-4
conv_model.compile(loss='categorical_crossentropy',
                   optimizer=optimizers.RMSprop(lr=LEARNING_RATE),
                   metrics=['acc'])

## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. You ask the model to make predictions about a test set—in this example, the `test_images` array. Verify that the predictions match the labels from the `test_labels` array.

To start training,  call the `model.fit` method—so called because it "fits" the model to the training data:

In [0]:

conv_model.fit(finished_images_array, finished_labels_array, epochs=5, batch_size=16)

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.88 (or 88%) on the training data.

## Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [0]:
conv_model.save("license_recognition_17.0.h5")

It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents *overfitting*. Overfitting is when a machine learning model performs worse on new, previously unseen inputs than on the training data.

## Make predictions

With the model trained, you can use it to make predictions about some images.

In [0]:
predictions = model.predict(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [0]:
predictions[0]

A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 10 different articles of clothing. You can see which label has the highest confidence value:

In [0]:
np.argmax(predictions[1])

So, the model is most confident that this image is an ankle boot, or `class_names[9]`. Examining the test label shows that this classification is correct:

In [0]:
test_labels[1]

Graph this to look at the full set of 10 class predictions.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(36), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Let's look at the 0th image, predictions, and prediction array. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percentage (out of 100) for the predicted label.

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Let's plot several images with their predictions. Note that the model can be wrong even when very confident.

In [0]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Finally, use the trained model to make a prediction about a single image.

In [0]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

`tf.keras` models are optimized to make predictions on a *batch*, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:

In [0]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

Now predict the correct label for this image:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(36), class_names, rotation=45)

`model.predict` returns a list of lists—one list for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [0]:
np.argmax(predictions_single[0])
print(class_names[14])

In [0]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

And the model predicts a label as expected.